In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import string
from tensorflow.python.framework import ops

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import RegexpParser
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stemmer = LancasterStemmer()


#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tahjy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tahjy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tahjy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load json and extract required data
with open('data.json') as json_data:
    symptoms = json.load(json_data)

# Looping through the Intents to Convert them to words, classes, documents and ignore_words.......
words = []
classes = []
documents = []
ignore_words = set(stopwords.words('english'))
for symptom in symptoms['intents']:
    pattern = symptom['patterns']
    words.extend(pattern)
    # add to documents in our corpus
    documents.append((pattern, symptom['tag']))
    # add to our classes list
    if symptom['tag'] not in classes:
        classes.append(symptom['tag'])

In [3]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

133 documents
132 classes ['abdominal_pain', 'abnormal_menstruation', 'acidity', 'acute_liver_failure', 'altered_sensorium', 'anxiety', 'back_pain', 'belly_pain', 'blackheads', 'bladder_discomfort', 'blister', 'blood_in_sputum', 'bloody_stool', 'blurred_and_distorted_vision', 'breathlessness', 'brittle_nails', 'bruising', 'burning_micturition', 'chest_pain', 'chills', 'cold_hands_and_feets', 'coma', 'congestion', 'constipation', 'continuous_feel_of_urine', 'continuous_sneezing', 'cough', 'cramps', 'dark_urine', 'dehydration', 'depression', 'diarrhoea', 'dischromic_patches', 'distention_of_abdomen', 'dizziness', 'drying_and_tingling_lips', 'enlarged_thyroid', 'excessive_hunger', 'extra_marital_contacts', 'family_history', 'fast_heart_rate', 'fatigue', 'fluid_overload', 'foul_smell_ofurine', 'headache', 'high_fever', 'hip_joint_pain', 'history_of_alcohol_consumption', 'increased_appetite', 'indigestion', 'inflammatory_nails', 'internal_itching', 'irregular_sugar_level', 'irritability', '

In [4]:
#Creating the Data for our Model
training = []
output = []

output_empty = [0] * len(classes)

#Creating Traning Set, Bag of Words for our Model

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


In [5]:
#Shuffling Randomly and Converting into Numpy Array for Faster Processing
random.shuffle(training)
training = np.array(training)

#Creating Train and Test Lists
train_x = list(training[:,0])
train_y = list(training[:,1])

#Building Neural Network for Out Chatbot to be Contextual
ops.reset_default_graph()

In [6]:
# define our Neural Network architecture using tflearn

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_symptoms_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
#Training the model
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

#Saving the model
model.save('model_symptoms.tflearn')

Training Step: 16999  | total loss: 0.00155 | time: 0.049s
| Adam | epoch: 1000 | loss: 0.00155 - acc: 1.0000 -- iter: 128/133
Training Step: 17000  | total loss: 0.00155 | time: 0.052s
| Adam | epoch: 1000 | loss: 0.00155 - acc: 1.0000 -- iter: 133/133
--
INFO:tensorflow:C:\Users\tahjy\Documents\Web Apps\asklepios_chatbot\model_symptoms.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# Save our training data which consists of the classes/tags, corpus
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data_symptoms", "wb" ) )

In [9]:
# Load our pickled training dataset
data = pickle.load( open( "training_data_symptoms", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('symptoms.json') as json_data:
    symptoms = json.load(json_data)
    
# load our saved model
model.load('./model_symptoms.tflearn')


INFO:tensorflow:Restoring parameters from C:\Users\tahjy\Documents\Web Apps\asklepios_chatbot\model_symptoms.tflearn


In [10]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

# Define our error threshold
ERROR_THRESHOLD = 0.20

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results)]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list[0]

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    return results

In [11]:
translate_table = dict((ord(char), None) for char in string.punctuation)   


        
while True:
    input_data = input("You- ").translate(translate_table)
    answer = response(input_data)
    print(answer)


KeyboardInterrupt: Interrupted by user